In [1]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import TextVectorization, Embedding, Dense, GlobalAveragePooling1D
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import LabelEncoder

In [2]:
# Data imports
train_file = pd.read_csv("train-data.tsv", sep='\t', header=None, names=['label', 'message'])
test_file = pd.read_csv("valid-data.tsv", sep='\t', header=None, names=['label', 'message'])
train_file

,label,message
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...
...,...,...
4174,ham,just woke up. yeesh its late. but i didn't fal...
4175,ham,what do u reckon as need 2 arrange transport i...
4176,spam,free entry into our £250 weekly competition ju...
4177,spam,-pls stop bootydelious (32/f) is inviting you ...


In [3]:
# Prepare the data
X_train = train_file['message'].values
y_train = train_file['label'].values

X_test = test_file['message'].values
y_test = test_file['label'].values

In [4]:

# Encode labels (ham=0, spam=1)
encoder = LabelEncoder()
y_train_enc = encoder.fit_transform(y_train)
y_test_enc = encoder.transform(y_test)

In [5]:
# Vectorization
max_tokens = 20000   # vocabulary size
output_len = 200     # max sequence length

vectorizer = TextVectorization(max_tokens=max_tokens, output_sequence_length=output_len)
vectorizer.adapt(X_train)

In [6]:
# Build Neural Network Model
inputs = keras.Input(shape=(1,), dtype=tf.string)              # raw text
x = vectorizer(inputs)                                        # vectorized -> (batch, maxlen)
x = layers.Embedding(input_dim=max_tokens, output_dim=64, mask_zero=True)(x)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(32))(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs, outputs)


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [7]:

# Train the model
history = model.fit(X_train, y_train_enc, 
                    validation_data=(X_test, y_test_enc),
                    epochs=5,
                    batch_size=16)

Epoch 1/5
262/262 ━━━━━━━━━━━━━━━━━━━━ 20s 69ms/step - accuracy: 0.9533 - loss: 0.1625 - val_accuracy: 0.9842 - val_loss: 0.0541
Epoch 2/5
262/262 ━━━━━━━━━━━━━━━━━━━━ 19s 72ms/step - accuracy: 0.9947 - loss: 0.0247 - val_accuracy: 0.9871 - val_loss: 0.0397
Epoch 3/5
262/262 ━━━━━━━━━━━━━━━━━━━━ 19s 72ms/step - accuracy: 0.9986 - loss: 0.0073 - val_accuracy: 0.9849 - val_loss: 0.0472
Epoch 4/5
262/262 ━━━━━━━━━━━━━━━━━━━━ 19s 72ms/step - accuracy: 0.9988 - loss: 0.0040 - val_accuracy: 0.9878 - val_loss: 0.0516
Epoch 5/5
262/262 ━━━━━━━━━━━━━━━━━━━━ 19s 72ms/step - accuracy: 0.9990 - loss: 0.0020 - val_accuracy: 0.9871 - val_loss: 0.0633


In [8]:
# Evaluate
loss, acc = model.evaluate(X_test, y_test_enc)
print(f"Test Accuracy: {acc:.4f}")

44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9871 - loss: 0.0633
Test Accuracy: 0.9871


In [9]:
# Predict function
def predict_message(message):
    input_text = tf.convert_to_tensor([message])
    proba = model.predict(input_text, verbose=0)[0][0]
    label = "spam" if proba >= 0.5 else "ham"
    return [float(proba), label]

In [10]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()

You passed the challenge. Great job!
